# Model Deployment with FastAPI

## Introduction



FASTAPI is a modern web framework designed for high-performance API development. It is ideal for deploying AI models due to its speed, type safety, and built-in documentation.



## Dataset Generation



To train our model, we generate synthetic data consisting of temperature, weather conditions, and day of the week.

In [11]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
import joblib

# generate synthetic data
data_size = 4000
np.random.seed(42)
temperature = np.random.uniform(-10, 40, data_size)  # Random temperatures in Celsius
weather_options = np.array(['rainy', 'clear', 'cloudy'])
weather = np.random.choice(weather_options, data_size)
weekday = np.random.randint(0, 7, data_size)  # 0=Sunday, 6=Saturday

# create DataFrame
df = pd.DataFrame({'temperature': temperature, 'weather': weather, 'weekday': weekday})



## Encoding Weather Feature


In [14]:
# define the encoder and fit it with possible values
weather_encoder = OneHotEncoder()
weather_encoded = weather_encoder.fit_transform(df[['weather']]).toarray()

# convert encoded values to DataFrame
encoded_columns = weather_encoder.get_feature_names_out(['weather'])
df_encoded = pd.DataFrame(data=weather_encoded, columns=encoded_columns)

# concatenate encoded features with original DataFrame
df = pd.concat([df, df_encoded], axis=1).drop(columns=['weather'])

# save the encoder
joblib.dump(weather_encoder, "weather_encoder.pkl")

['weather_encoder.pkl']


## Model Training


In [15]:
# define features and target
X = df[['temperature', 'weather_rainy', 'weather_clear', 'weather_cloudy', 'weekday']]
y = X['temperature'] * 0.3 + X['weather_rainy'] * 5 - X['weather_clear'] * 2 + X['weekday'] * 0.1 + np.random.randn(data_size) * 2  # Example target

# train a simple linear regression model
model = LinearRegression()
model.fit(X, y)

# save the model
joblib.dump(model, "model.pkl")

['model.pkl']


## Introduction to Joblib



Joblib is a Python library used for efficiently saving and loading machine learning models and preprocessors.



## Loading the Model and Encoder



In [16]:
import joblib

# load pre-trained model and encoder
model = joblib.load("model.pkl")
weather_encoder = joblib.load("weather_encoder.pkl")



## Introduction to Pydantic



Pydantic is used for data validation in FastAPI. It ensures strict data structures for API requests.



### Defining Model Input Schema



In [17]:
from pydantic import BaseModel
from typing import Literal

class ModelInput(BaseModel):
    temperature: float  # Temperature in Celsius
    weather: Literal['rainy', 'clear', 'cloudy']  # Categorical weather condition
    weekday: int  # Integer representing the day of the week (0=Sunday, 6=Saturday)

## Creating FastAPI Endpoint



To start, we create a simple FastAPI endpoint to return a basic message.



### Full Code for Simple Example



Save the following code as `main.py`:



In [21]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def home():
    return {"message": "FastAPI is running!"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000, reload=True)

SystemExit: 1

/Users/rahafmasmali/Desktop/T2/templets/fastAPI/venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Running the API with Uvicorn

```bash
uvicorn main:app --host 127.0.0.1 --port 8000 --reload
```

### Testing with cURL

```bash
curl -X GET "http://127.0.0.1:8000/"
```

### Testing with Python Requests

In [25]:
import requests

url = "http://127.0.0.1:8000/"
response = requests.get(url)
print(response.json())

{'detail': 'Not Found'}


## Deployment Strategies



- **Docker**: Containerizing the API for portability.
- **Cloud Services**: Deploying on AWS, GCP, or Azure.
- **Serverless**: Using platforms like AWS Lambda or Google Cloud Run.
- **Render**: Deploying on Render for simplicity and ease of use.



## Deploying on Render

### Steps to Deploy


1. **Create a `requirements.txt` file**

```txt
fastapi
uvicorn
joblib
scikit-learn
numpy
pandas
```

2. **Create a `start.sh` script**

```bash
#!/bin/bash
uvicorn main:app --host 127.0.0.1 --port $PORT
```

3. **Push the code to GitHub**
4. **Go to [Render](https://render.com/) and create a new Web Service**
5. **Connect your GitHub repository**
6. **Set the Build Command**

```bash
pip install -r requirements.txt
```

7. **Set the Start Command**

```bash
bash start.sh
```

8. **Deploy and test your API**



## Exercise: Implement the Full Model Endpoint

Below is a skeleton structure to guide your implementation. Complete the script using the knowledge from previous sections.



In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Literal
import joblib

# define input schema
#...

# initialize FastAPI
#...

# load model and encoder
#...

# define prediction endpoint
@app.post("/predict")
def predict(data: ModelInput):
    # encode categorical input
    #...
    
    # create feature vector
    #...
    
    # generate prediction
    #...
    
    return {"prediction": None}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000, reload=True)

### Running the Full Model Endpoint


```bash
uvicorn main:app --host {Render_URL} --port {Render_Port} --reload
```


### Testing with cURL


```bash
curl -X POST "http://{Render_URL}:{Render_Port}/predict" \
     -H "Content-Type: application/json" \
     -d '{"temperature": 25.0, "weather": "clear", "weekday": 2}'
```


### Testing with Python Requests in deployed Streamlit app

In [ ]:
import requests

#... other code ...

# use streamlit inputs here
url = "http://{Render_URL}:{Render_Port}/predict"
data = {
    "temperature": None, # streamlit input here
    "weather": None, # streamlit input here
    "weekday": None # streamlit input here
}
response = requests.post(url, json=data)
print(response.json())

#... other code ...